# Anomalous Financial Transaction Detection

본 대회의 과제는 금융 거래 데이터에서 **이상 거래를 탐지하는 기능**을 개선하고 활용도를 높이는 분류 AI모델을 개발하는 것입니다.

특히, 클래스 불균형 문제를 해결하기 위해 오픈소스 생성형 AI 모델을 활용하여 부족한 클래스의 데이터를 보완하고, 이를 통해 분류 모델의 성능을 향상시키는 것이 핵심 목표입니다.

이러한 접근을 통해 금융보안에 특화된 데이터 분석 및 활용 역량을 강화하여 전문 인력을 양성하고, 금융권의 AI 활용 어려움에 따른 해결 방안을 함께 모색하며 금융 산업의 AI 활용 활성화를 지원하는 것을 목표로 합니다.


# 전략

## 1. 좋은 학습 데이터 생성(생성형 AI)

- 좋은 생성 데이터 : 원본 데이터랑 겹치지 않는 데이터 (특정 열의 값이 같은 경우, 나머지 값들도 같은 경우가 적은 상태)
    - 생성형 AI 모델 비교 및 선택
        - ctgan, tvae
        - ctab-ga, g-ctgan, vaem : 논문 참고해 구현 필요

    - 생성형 AI 데이터 비율 조정해보기

## 2. 이상치 처리

- z-score
    - 다른 분포로 가정해보기 (KDE로 분포 찾기?)

## 3. 좋은 분류 모델

xgboost - baseline

- 딥러닝 모델 :  tabnet(특성 중요도 표시 가능), wide&deep(추천시스템)
- 로지스틱 회귀
- 다양한 분류 모델 짬뽕 (multiple classifier system)
- 머신러닝 모델 : xgboost, adaboost, ...

## 4.  특성 공학

- 데이터 처리
    - 데이터 정규화 (z-score 이용하는게 좋은 듯. test 데이터에서 이상치 발생할 수 있으므로)

- 특성 재조립
    - 마지막 영업점 / ATM 거래 일자 차이
    - 거래발생 위치 -> 시군구 분리 후 특성으로 활용
    - 범주의 개수가 3개 이상인 데이터들은 ordinal 이 아닌 군집화 후 one hot vector 로 변환

## 5. 기타 금융 이상거래 도메인 지식

- 특징?

In [ ]:
!pip install sdv
!pip install xgboost
!pip install optuna
!pip install autogluon
!pip install holidays
!pip install catboost

In [ ]:
# Import Library
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import f1_score

# 분류 모델
from xgboost import XGBClassifier
from catboost import CatBoostClassifier #GPU training
from lightgbm import LGBMClassifier #GPU training

# 모델 저장
import joblib

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sklearn.preprocessing import KBinsDiscretizer


# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# Model optimizing
from sklearn.feature_selection import RFECV

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice



# 데이터 전처리

In [ ]:

# 생성 🏭
# Load Data
train_all = pd.read_csv("./train.csv")
test_all = pd.read_csv("./test.csv")
train = train_all.drop(columns="ID")
train["Fraud_Type"].value_counts()
'''
(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.
(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.
(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.
'''

'\n(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.\n(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.\n(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.\n'

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 63 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Customer_Birthyear                             120000 non-null  int64  
 1   Customer_Gender                                120000 non-null  object 
 2   Customer_personal_identifier                   120000 non-null  object 
 3   Customer_identification_number                 120000 non-null  object 
 4   Customer_registration_datetime                 120000 non-null  object 
 5   Customer_credit_rating                         120000 non-null  object 
 6   Customer_flag_change_of_authentication_1       120000 non-null  int64  
 7   Customer_flag_change_of_authentication_2       120000 non-null  int64  
 8   Customer_flag_change_of_authentication_3       120000 non-null  int64  
 9   Customer_flag_change_of_authenticatio

In [ ]:
obj_cat_columns=[col for col in train.select_dtypes(['object']).columns if train[col].nunique() <= 20]
obj_idf_columns=[col for col in train.select_dtypes(['object']).columns if train[col].nunique() > 20]
num_cat_columns=[col for col in train.select_dtypes(['int64', 'float64']).columns if train[col].nunique() <= 2]
num_con_columns=[col for col in train.select_dtypes(['int64', 'float64']).columns if train[col].nunique() > 2]

obj_cat_columns.remove('Fraud_Type')

In [ ]:
print('obj_cat_columns : ', obj_cat_columns)
print('obj_idf_columns : ', obj_idf_columns)
print('num_cat_columns : ', num_cat_columns)
print('num_con_columns : ', num_con_columns)

obj_cat_columns :  ['Customer_Gender', 'Customer_credit_rating', 'Customer_loan_type', 'Account_account_type', 'Channel', 'Operating_System', 'Error_Code', 'Type_General_Automatic', 'Access_Medium']
obj_idf_columns :  ['Customer_personal_identifier', 'Customer_identification_number', 'Customer_registration_datetime', 'Account_account_number', 'Account_creation_datetime', 'Transaction_Datetime', 'IP_Address', 'MAC_Address', 'Location', 'Recipient_Account_Number', 'Time_difference', 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date']
num_cat_columns :  ['Customer_flag_change_of_authentication_1', 'Customer_flag_change_of_authentication_2', 'Customer_flag_change_of_authentication_3', 'Customer_flag_change_of_authentication_4', 'Customer_rooting_jailbreak_indicator', 'Customer_mobile_roaming_indicator', 'Customer_VPN_Indicator', 'Customer_flag_terminal_malicious_behavior_1', 'Customer_flag_terminal_malicious_behavior_2', 'Customer_flag_term

In [ ]:
import holidays

def is_holiday(date):
    kr_holidays=holidays.KR(years=date.year)
    if date in kr_holidays:
        return 1
    else:
        return 0

def timezone(date):
    if date.hour>=1 and date.hour<=6:
        return 'dawn'

    elif date.hour>=7 and date.hour<=12:
        return 'morning'

    elif date.hour>=13 and date.hour<=18:
        return 'evening'

    else:
        return 'night'

def feature_engineering(df):

  # Time_difference 컬럼을 총 초로 변환 및 이상치 처리
  df['Time_difference_seconds'] = pd.to_timedelta(df['Time_difference']).dt.total_seconds()

  df['Transaction_Datetime']=pd.to_datetime(df['Transaction_Datetime'])
  df['Transaction_Datetime_isholiday']=df['Transaction_Datetime'].apply(is_holiday)
  df['Transaction_Datetime_day']=df['Transaction_Datetime'].apply(lambda x: x.strftime("%A"))
  df['Transaction_Datetime_timezone']=df['Transaction_Datetime'].apply(timezone)

  df['Location_District_Size']=df['Location'].apply(lambda x: 0 if x.split()[0][-1]=="도" else 1)
  df['Location_District']=df['Location'].apply(lambda x: x.split()[0])
  df['Account_after_transaction']=df['Account_balance']-df['Account_initial_balance']

  df['Last_bank_branch_transaction_datetime']=pd.to_datetime(df['Last_bank_branch_transaction_datetime'])
  df['Last_atm_transaction_datetime']=pd.to_datetime(df['Last_atm_transaction_datetime'])
  df['Last_atm_bank_transaction_time_difference']=(df['Last_bank_branch_transaction_datetime']-df['Last_atm_transaction_datetime']).dt.total_seconds()

  df['Customer_malicious_behavior']=(df['Customer_rooting_jailbreak_indicator']+
                                      df['Customer_mobile_roaming_indicator']+
                                      df['Customer_VPN_Indicator']+
                                      df['Customer_flag_terminal_malicious_behavior_1']+
                                      df['Customer_flag_terminal_malicious_behavior_2']+
                                      df['Customer_flag_terminal_malicious_behavior_3']+
                                      df['Customer_flag_terminal_malicious_behavior_4']+
                                      df['Customer_flag_terminal_malicious_behavior_5']+
                                      df['Customer_flag_terminal_malicious_behavior_6'])

  df['Customer_change_of_authentication']=(df['Customer_flag_change_of_authentication_1']+
                                              df['Customer_flag_change_of_authentication_2']+
                                              df['Customer_flag_change_of_authentication_3']+
                                              df['Customer_flag_change_of_authentication_4'])

  df['Account_change']=(df['Account_indicator_release_limit_excess']+
                          df['Account_release_suspention']+
                          df['Flag_deposit_more_than_tenMillion']+
                          df['Unused_account_status']+
                          df['Recipient_account_suspend_status'])

  return df



In [ ]:
train=feature_engineering(train)

In [ ]:

obj_cat_columns=obj_cat_columns+['Location_District', 'Transaction_Datetime_timezone', 'Transaction_Datetime_day']
num_cat_columns=num_cat_columns+['Transaction_Datetime_isholiday', 'Location_District_Size']
num_con_columns=num_con_columns+['Last_atm_bank_transaction_time_difference','Time_difference_seconds', 'Account_after_transaction',
                'Account_change','Customer_change_of_authentication','Customer_malicious_behavior']


# KNN 군집 데이터 추가

# obj_idf_columns  에서 가공할 열 :
# 'Customer_registration_datetime', 'Account_creation_datetime', 'Transaction_Datetime', : 거래 시간(밤,낮인지), 거래 일자(공휴일 혹은 평일)
# 'Location', : 이상 거래가 자주 발생하는 장소 - 시군구로 나누고 시각화해보기
# 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date' : 연도-월로 나눠 가장 최근으로부터 얼마나 떨어져 있는지 확인

In [ ]:
print('obj_cat_columns : ', obj_cat_columns)
print('obj_idf_columns : ', obj_idf_columns)
print('num_cat_columns : ', num_cat_columns)
print('num_con_columns : ', num_con_columns)

obj_cat_columns :  ['Customer_Gender', 'Customer_credit_rating', 'Customer_loan_type', 'Account_account_type', 'Channel', 'Operating_System', 'Error_Code', 'Type_General_Automatic', 'Access_Medium', 'Location_District', 'Transaction_Datetime_timezone', 'Transaction_Datetime_day']
obj_idf_columns :  ['Customer_personal_identifier', 'Customer_identification_number', 'Customer_registration_datetime', 'Account_account_number', 'Account_creation_datetime', 'Transaction_Datetime', 'IP_Address', 'MAC_Address', 'Location', 'Recipient_Account_Number', 'Time_difference', 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date']
num_cat_columns :  ['Customer_flag_change_of_authentication_1', 'Customer_flag_change_of_authentication_2', 'Customer_flag_change_of_authentication_3', 'Customer_flag_change_of_authentication_4', 'Customer_rooting_jailbreak_indicator', 'Customer_mobile_roaming_indicator', 'Customer_VPN_Indicator', 'Customer_flag_terminal_malicio

In [ ]:
model_fit_columns=obj_cat_columns+num_cat_columns+num_con_columns
# model_fit_columns=[x for x in model_fit_columns if x not in ['ID', 'Fraud_Type']]

In [ ]:
model_fit_columns

['Customer_Gender',
 'Customer_credit_rating',
 'Customer_loan_type',
 'Account_account_type',
 'Channel',
 'Operating_System',
 'Error_Code',
 'Type_General_Automatic',
 'Access_Medium',
 'Location_District',
 'Transaction_Datetime_timezone',
 'Transaction_Datetime_day',
 'Customer_flag_change_of_authentication_1',
 'Customer_flag_change_of_authentication_2',
 'Customer_flag_change_of_authentication_3',
 'Customer_flag_change_of_authentication_4',
 'Customer_rooting_jailbreak_indicator',
 'Customer_mobile_roaming_indicator',
 'Customer_VPN_Indicator',
 'Customer_flag_terminal_malicious_behavior_1',
 'Customer_flag_terminal_malicious_behavior_2',
 'Customer_flag_terminal_malicious_behavior_3',
 'Customer_flag_terminal_malicious_behavior_4',
 'Customer_flag_terminal_malicious_behavior_5',
 'Customer_flag_terminal_malicious_behavior_6',
 'Customer_inquery_atm_limit',
 'Customer_increase_atm_limit',
 'Account_indicator_release_limit_excess',
 'Account_indicator_Openbanking',
 'Account_rel

# EDA

# 데이터 생성

In [ ]:
fraud_types = train['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

N_CLS_PER_GEN = 1000
N_SAMPLE = 100

train_new = train_all.drop(columns="ID")
train_new['Time_difference_seconds'] = pd.to_timedelta(train_new['Time_difference']).dt.total_seconds()

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):

    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train_new[train_new["Fraud_Type"] == fraud_type]

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=42)

    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)

    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )

    synthesizer = CTGANSynthesizer(
                            metadata,
                            epochs=100
                        )
    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)

    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')

    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)

    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)



100%|██████████| 13/13 [04:50<00:00, 22.36s/it]


# Optuna를 이용한 fine tuning


In [ ]:
# 최종 결과 확인
all_synthetic_data_feature_engineered=all_synthetic_data.copy()

all_synthetic_data_feature_engineered=feature_engineering(all_synthetic_data_feature_engineered)

## 원본 데이터와 concat

train_total = pd.concat([train, all_synthetic_data_feature_engineered])
print(train_total.shape)


(133000, 74)


In [ ]:

# 불균형 범주형 데이터 교차검증
fold_num=5
f1_macro_sum=0
skf=StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=42)


# 범주형 변수 인코딩

ct=ColumnTransformer([('encoder', OneHotEncoder(sparse=False, drop='first',handle_unknown='ignore'),
                        obj_cat_columns+num_cat_columns),
                    ('scaler', StandardScaler(), num_con_columns),
                    ('binning', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile'), num_con_columns)]
                          )

In [ ]:

# 테스트 데이터 인코딩
test_x=test_all.copy()
test_x=feature_engineering(test_x)

In [ ]:
test_x=test_x[model_fit_columns]

ct.fit(train)
test_x_encoded_array=ct.transform(test_x)

# 특성 순서 맞추기 및 데이터 타입 일치
column_names=ct.get_feature_names_out()
test_x_encoded=pd.DataFrame(test_x_encoded_array, columns=column_names)

train_total_x=train_total[model_fit_columns]
train_total_x_array=ct.fit_transform(train_total_x)

column_names=ct.get_feature_names_out()
train_total_x_encoded=pd.DataFrame(train_total_x_array, columns=column_names)


train_total_y=train_total['Fraud_Type']

le_subclass = LabelEncoder()
train_total_y_encoded = le_subclass.fit_transform(train_total_y)

In [ ]:
train_total_x_encoded.to_csv('train_total_x_encoded.csv', index=False)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

def autogluon_validate_score(data, cv=skf, label='Fraud_Type', presets='best_quality', time_limit=10800, excluded_model_types=['CAT']):

  train_data=TabularDataset(data)

  predictor=TabularPredictor(label='Fraud_Type', eval_metric='f1_macro').fit(train_data, presets=presets, time_limit=time_limit, excluded_model_types=excluded_model_types)

  prediction_leaderboard=pd.DataFrame()
  prediction_leaderboard=predictor.leaderboard()

  print(prediction_leaderboard)
  prediction_leaderboard.sort_values(by=['score_val', 'fit_time'], ascending=False, inplace=True)
  models_to_keep=list(prediction_leaderboard.loc[:10, 'model'].values)

  predictor.delete_models(models_to_keep=models_to_keep)
  predictor.refit_full()
  models = predictor.info()['model_performance'].index
  probs_df = pd.DataFrame()

  for model in models:
      # 각 모델에 대한 예측 확률값 계산
      model_probs = predictor.predict_proba(test_data, model=model)
      probs_df[f'model'] = model_probs

  predictor.save()

  return probs_df

In [ ]:

train_total_autogluon=pd.DataFrame()

for i, (train_index, valid_index) in enumerate(skf.split(train_total, train_total['Fraud_Type'])):
  train_total_autogluon=pd.concat([train_total_autogluon, train_total.iloc[valid_index]], axis=0)

train_total_autogluon.reset_index(inplace=True)
train_total_autogluon.drop(columns=['index'], inplace=True)
train_total_autogluon=feature_engineering(train_total_autogluon)
train_total_autogluon[model_fit_columns]=ct.fit_transform(train_total_autogluon[model_fit_columns])
train_total_autogluon.to_csv('train_total_autogluon.csv', index=False)

test_autogluon
test_autogluon=feature_engineering(test_autogluon)
test_autogluon=ct.transform(test_autogluon[model_fit_columns])

In [ ]:
train_total_autogluon=pd.read_csv('train_total_autogluon.csv')

In [ ]:
probs_df=autogluon_validate_score(train_total_autogluon[model_fit_columns+['Fraud_Type']])

No path specified. Models will be saved in: "AutogluonModels\ag-20240826_150355"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.17
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Memory Avail:       5.93 GB / 15.93 GB (37.2%)
Disk Space Avail:   141.96 GB / 953.22 GB (14.9%)
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels\ag-20240826_150355"
Train Data Rows:    133000
Train Data Columns: 73
Label Column:       Fraud_Type
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 13) unique label values:  ['m', 'k', 'h', 'e', 'f', 'i', 'g', 'b', 'j', 'a']
	If 'multiclass' is not the correct problem_type, ple

                  model  score_val eval_metric  pred_time_val    fit_time  \
0   WeightedEnsemble_L2   0.964160    f1_macro       0.168038  366.387801   
1              LightGBM   0.961960    f1_macro       0.038008   19.354337   
2       NeuralNetFastAI   0.957218    f1_macro       0.056013  330.236780   
3         LightGBMLarge   0.955997    f1_macro       0.075017   43.122663   
4               XGBoost   0.952965    f1_macro       0.072016   16.406677   
5            LightGBMXT   0.951972    f1_macro       0.079018   32.656822   
6        NeuralNetTorch   0.939334    f1_macro       0.058013  131.448836   
7      RandomForestGini   0.930497    f1_macro       0.072118   50.754940   
8      RandomForestEntr   0.925980    f1_macro       0.074029   34.333378   
9        ExtraTreesGini   0.848078    f1_macro       0.072021   54.578231   
10       ExtraTreesEntr   0.821360    f1_macro       0.074020   38.187823   

    pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0 

	Stopping at the best epoch learned earlier - 14.
	169.88s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMXT_FULL ...
	22.5s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_FULL ...
	12.39s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestGini_FULL ...
	53.4s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestEntr_FULL ...
	36.3s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: ExtraTreesGini_FULL ...
	56.59s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: ExtraTreesEntr_FULL ...
	39.55s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: XGBoost_FULL ...
	11.24s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: NeuralNetTorch_FULL ...
	50.67s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMLarge_FULL ...
	27.84s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'

In [ ]:
probs_df.to_csv('probs_df.csv')

# Model Stacking

In [ ]:
#parameters suggested for meta model
meta_model_params = {
    'n_estimators': 47,
    'alpha': 6.422755620546236e-05,
    'subsample': 0.8452333586225941,
    'colsample_bytree': 0.7651776055349394,
    'max_depth': 3,
    'min_child_weight': 8,
    'learning_rate': 0.011014344390319484,
    'gamma': 6.1495867050966066e-06
}

meta_model = XGBClassifier(**meta_model_params, random_state=42)

In [ ]:
from sklearn.feature_selection import RFECV

min_features_to_select = 1

#Selection
pipeline = Pipeline([
    ('rfecv', RFECV(estimator=meta_model,
                    step=1,
                    cv=skf,
                    scoring="f1_macro",
                    min_features_to_select=min_features_to_select,
                    n_jobs=-1,))
])

# Fit the pipeline on the training data
pipeline.fit(probs_df, train_total_y_encoded)

#CV score
print("Best CV score: ")
selected_features = np.array( probs_df.columns)[pipeline.named_steps['rfecv'].support_]
print( pipeline.named_steps['rfecv'].cv_results_["mean_test_score"][len(selected_features) - 1])



# Selected features after RFECV
print('Number of evaluated features:', len(probs_df.columns))
print('Number of selected features:', len(selected_features))
print("Selected Features:", selected_features)

ValueError: could not convert string to float: 'm'

In [ ]:
final_predictor=predictor.load()
models = final_predictor.info()['model_performance'].index
  submission_probs_df = pd.DataFrame()

for model in models:
    # 각 모델에 대한 예측 확률값 계산
    model_probs = final_predictor.predict_proba(test_autogluon, model=model)
    submission_probs_df[f'model'] = model_probs

In [ ]:
meta_model = meta_model.fit(probs_df[selected_features],  train_total_y_encoded)
preds_test =  meta_model.predict(submission_probs_df[selected_features])

predictions_label = le_subclass.inverse_transform(preds_test)

In [ ]:
# 분류 예측 결과 제출 데이터프레임(DataFrame)

clf_submission = pd.read_csv("./sample_submission.csv")
clf_submission["Fraud_Type"] = prediction_label

# 폴더 생성 및 작업 디렉토리 변경
os.makedirs('./submission', exist_ok=True)
os.chdir("./submission/")

# CSV 파일로 저장
clf_submission.to_csv('./clf_submission.csv', encoding='UTF-8-sig', index=False)
all_synthetic_data.to_csv('./syn_submission.csv', encoding='UTF-8-sig', index=False)

# ZIP 파일 생성 및 CSV 파일 추가
with zipfile.ZipFile("../baseline_submission_.zip", 'w') as submission:
    submission.write('clf_submission.csv')
    submission.write('syn_submission.csv')
